In [1]:
import pandas as pd
import os
absolutepath = os.path.abspath(os.getcwd())
fileDirectory = os.path.dirname(absolutepath)
parentDirectory = os.path.dirname(fileDirectory)
dataprocDirectory = os.path.join(fileDirectory, 'data/proc')
datarawDirectory = os.path.join(fileDirectory, 'data/raw')

# load treament status

In [2]:
authid_level_dataset = pd.read_csv(os.path.join(dataprocDirectory, 
                                                'author_treatment_assignment_stacked.csv'))

In [3]:
authid_level_dataset.is_jttp.mean()

np.float64(0.0)

# load outcome
# keep only restricted
\copy (select * from scopus_authors_year_outcome_chinese_deduped left join all_authors_in_restriction_jttp on scopus_authors_year_outcome_chinese_deduped.authid = all_authors_in_restriction_jttp.authid where all_authors_in_restriction_jttp.restriction = 1) to '/cluster/work/lawecon/Work/dcai/jiycai/thousand_talent/thousand_talent/scripts/10282022_iteration_peer_effect_stacking/data/proc/author_pool_outcomes_restricted.csv' csv header;

In [5]:
outcome_dataset = pd.read_csv(os.path.join(dataprocDirectory, 
                                                'author_pool_outcomes_restricted.csv'))

# for each scholar create panel between 2005 and 2019

In [6]:
def create_balanced_panel():
    print('creating min max')
    outcome_dataset_authid_max_min = outcome_dataset[['authid','restriction','year']].groupby(['authid','restriction']).\
    agg({'year':['min','max']}).reset_index()
    outcome_dataset_authid_max_min.columns = ['_'.join(col) for col in outcome_dataset_authid_max_min.columns]
    outcome_dataset_authid_max_min.columns = ['authid','restriction','min_year','max_year']
    # merge with min max
    authid_level_merged = authid_level_dataset.merge(outcome_dataset_authid_max_min, 
                                                      how = 'left',
                                                      on = 'authid')
    authid_level_merged['career_length'] = authid_level_merged.max_year-authid_level_merged.min_year+1
    
    print('create skeleton')
    import itertools
    auth_year = [(v[0][0],v[0][1],v[1]) for v in \
                 list(itertools.product(authid_level_dataset[['authid','cohort']].values,range(2005,2020)))]
    data_skeleton = pd.DataFrame(auth_year, columns = ['authid','cohort','year'])
    data_balanced = data_skeleton.merge(authid_level_merged, how = 'left', on = ['authid','cohort'])
    data_balanced['before_first_year'] = 1*(data_balanced.year < data_balanced.min_year)
    data_balanced['after_last_year'] = 1*(data_balanced.year > data_balanced.max_year)
    
    print('merge in outcome')
    data_balanced = data_balanced.merge(outcome_dataset, 
                                    how = 'left', 
                                    on = ['authid','year'])
    
    print('impute missing')
    for var in ['citedby_count','cite_split','frac_count_docs',
        'num_docs', 'if_top10', 'if_top50', 'if_lower50', 'num_ranked',
        'num_docs_ex_j','num_docs_intl','num_docs_intl_ex_j','funded','sum_citescore']:
        data_balanced[var] = data_balanced[var].fillna(0)
        
    print('drop not yet or already treated')
    data_balanced = data_balanced[(data_balanced.treatment_status==1)|
                                  #((data_balanced.treatment_status==2)&(data_balanced.year<data_balanced.treat_year))|
                                  (data_balanced.treatment_status==3)]
    print('keep max_year > cohort')
    data_balanced = data_balanced[data_balanced.max_year > data_balanced.cohort]
    
    return data_balanced

In [7]:
balanced_dataset = create_balanced_panel()

creating min max
create skeleton
merge in outcome
impute missing
drop not yet or already treated
keep max_year > cohort


In [8]:
balanced_dataset.to_csv(os.path.join(dataprocDirectory, 
                                                'balanced_panel_wo_affiliation_never_treated_control_group.csv'),
                        index=False)